# Variational Autoencoder Keras Implementation

In [1]:
import os
import numpy as np
import keras.backend as K
from keras.models import Model, Input
from keras.layers import Dense, LeakyReLU, Lambda, Conv2D, BatchNormalization
from keras.losses import mean_squared_error

Using TensorFlow backend.


### Define the VAE

In [2]:
# The sampling layer
def sampling(z):
    z_mean, z_log_var = z
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [9]:
# The encoder template
def Encoder():
    # You can modify the architecture for your own design
    # The following is just a simple example
    x = Dense(512)
    x = LeakyReLU()(x)
    
    # The sampling layer
    z_mean = Dense(128, name='z_mean')(x)
    z_log_var = Dense(128, name='z_log_var')(x)
    
    return z_mean, z_log_var 

In [4]:
# The decoder template
def Decoder(x):
    # You can modify the architecture for your own design
    # The following is just a simple example
    
    x = Dense(512)(x)
    x = LeakyReLU()(x)
    
    x = Dense(1024)(x)
    x = LeakyReLU()(x)
    
    outputs = x
    
    return outputs

In [5]:
# define KL loss
def KL_Loss(z_mean, z_log_var):
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    print(kl_loss)
    
    return kl_loss

In [6]:
# define reconstruction loss
def Reconstruction_Loss(inputs, outputs):
    loss = mean_squared_error(inputs, outputs)
    return loss

In [7]:
# The full autoencoder
def Autoencoder():
    inputs = Input((1024,))
    z_mean, z_log_var = Encoder()(inputs)
    z = Lambda(sampling, name='z')([z_mean, z_log_var])
    outputs = Decoder(z)
    
    # You can modify the following for your own design
    beta = 1.0
    
    VAE = Model(inputs, outputs)
    
    VAE_Loss = K.mean(Reconstruction_Loss(inputs, outputs) * 1024 + KL_Loss(z_mean, z_log_var) * beta)
    VAE.add_loss(VAE_Loss)
    VAE.compile(optimizer='adam')
    
    Encoder_ = Model(inputs, z)
    
    return VAE, Encoder_

### Build the VAE

In [10]:
VAE, Encoder_ = Autoencoder()
print(VAE.summary())

ValueError: Layer leaky_re_lu_1 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.layers.core.Dense'>. Full input: [<keras.layers.core.Dense object at 0x000001EFBEA67EF0>]. All inputs to the layer should be tensors.

### Generate some toy data

In [9]:
Data = np.random.rand(100, 1024)
print(Data)

[[0.49661779 0.79254345 0.31734548 ... 0.74377957 0.77536782 0.07223718]
 [0.14895057 0.04332683 0.96541642 ... 0.53211502 0.25367769 0.99844478]
 [0.28139611 0.5437912  0.40258143 ... 0.98637925 0.72418104 0.14062972]
 ...
 [0.80646686 0.06094832 0.18961759 ... 0.94441179 0.9026205  0.30932427]
 [0.48568428 0.06456302 0.39074083 ... 0.70393537 0.82346195 0.92771353]
 [0.57645433 0.27057066 0.18872184 ... 0.74669469 0.74966826 0.66129119]]


### Fit the VAE

In [35]:
VAE.fit(Data, epochs=500)

Epoch 1/500
100/100 [==============================] - 0s 219us/step - loss: 41.4665
Epoch 2/500
100/100 [==============================] - 0s 209us/step - loss: 42.2921
Epoch 3/500
100/100 [==============================] - 0s 240us/step - loss: 42.3233
Epoch 4/500
100/100 [==============================] - 0s 259us/step - loss: 42.7681
Epoch 5/500
100/100 [==============================] - 0s 249us/step - loss: 45.0777
Epoch 6/500
100/100 [==============================] - 0s 230us/step - loss: 43.9761
Epoch 7/500
100/100 [==============================] - 0s 249us/step - loss: 46.5115
Epoch 8/500
100/100 [==============================] - 0s 239us/step - loss: 44.3698
Epoch 9/500
100/100 [==============================] - 0s 229us/step - loss: 44.1891
Epoch 10/500
100/100 [==============================] - 0s 229us/step - loss: 42.2108
Epoch 11/500
100/100 [==============================] - 0s 229us/step - loss: 43.3361
Epoch 12/500
100/100 [==============================] - 0s 240u

100/100 [==============================] - 0s 249us/step - loss: 41.5444
Epoch 191/500
100/100 [==============================] - 0s 269us/step - loss: 40.0029
Epoch 192/500
100/100 [==============================] - 0s 269us/step - loss: 38.8655
Epoch 193/500
100/100 [==============================] - 0s 239us/step - loss: 38.7719
Epoch 194/500
100/100 [==============================] - 0s 259us/step - loss: 39.5678
Epoch 195/500
100/100 [==============================] - 0s 259us/step - loss: 39.2500
Epoch 196/500
100/100 [==============================] - 0s 279us/step - loss: 40.2528
Epoch 197/500
100/100 [==============================] - 0s 280us/step - loss: 40.6249
Epoch 198/500
100/100 [==============================] - 0s 249us/step - loss: 39.2656
Epoch 199/500
100/100 [==============================] - 0s 229us/step - loss: 41.2482
Epoch 200/500
100/100 [==============================] - 0s 259us/step - loss: 40.4508
Epoch 201/500
100/100 [==============================] - 

100/100 [==============================] - 0s 249us/step - loss: 40.4251
Epoch 379/500
100/100 [==============================] - 0s 229us/step - loss: 40.8443
Epoch 380/500
100/100 [==============================] - 0s 229us/step - loss: 40.4692
Epoch 381/500
100/100 [==============================] - 0s 259us/step - loss: 40.6406
Epoch 382/500
100/100 [==============================] - 0s 219us/step - loss: 41.6440
Epoch 383/500
100/100 [==============================] - 0s 259us/step - loss: 42.1810
Epoch 384/500
100/100 [==============================] - 0s 219us/step - loss: 39.9849
Epoch 385/500
100/100 [==============================] - 0s 259us/step - loss: 40.1582
Epoch 386/500
100/100 [==============================] - 0s 249us/step - loss: 40.7426
Epoch 387/500
100/100 [==============================] - 0s 269us/step - loss: 38.7563
Epoch 388/500
100/100 [==============================] - 0s 249us/step - loss: 40.7197
Epoch 389/500
100/100 [==============================] - 

### Save model

In [11]:
Encoder_.save("VAE_Encoder_Keras.hdf5")
VAE.save("VAE_Keras.hdf5")